In [1]:
#!pip install mlflow==2.1.0 requests
#!pip install pyngrok

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM

import mlflow
import os
from IPython.display import display
import tensorflow as tf

In [3]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

In [4]:
token = "2QiqRXPGx995tJ7wDJQeofnMlXJ_2h8Rnbtfm2mJCK3h8ro8B" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

In [5]:
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

<NgrokTunnel: "https://a624-34-121-80-186.ngrok-free.app" -> "http://localhost:5000">

In [7]:
mlflow.set_tracking_uri("http://localhost:5000")

In [9]:
exp_id = mlflow.create_experiment(name="MLDS61", artifact_location="mlruns/")

In [10]:
df = pd.read_csv("https://raw.githubusercontent.com/FaSaSu20/MLD6_Proy/master/src/nombre_paquete/database/data.csv") # Cargo otros datos parecidos
df["Fecha"] = pd.to_datetime(df['Fecha'].astype(str), format ='%Y-%m-%d %H:%M:%S')
df = df[["Fecha" ,"AGUA", "CARBON", "GAS", "LIQUIDOS", "PPP Bolsa diario"]]

x = df[["AGUA", "CARBON", "GAS", "LIQUIDOS"]].to_numpy()
y = df[["PPP Bolsa diario"]].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle = False)

In [11]:
with mlflow.start_run(
        run_name="MLDS",
        experiment_id=exp_id
        ):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    model.fit(X_train, y_train, epochs = 50, verbose = 0)
    y_pred = model.predict(X_test)
    #mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.tensorflow.log_model(model, "model")

17/17 [==============================] - 1s 3ms/step


2023/06/28 01:36:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2023/06/28 01:37:08 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.4.10+cuda11.cudnn86) contains a local version label (+cuda11.cudnn86). MLflow logged a pip requirement for this package as 'jaxlib==0.4.10' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [12]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"

In [13]:
command = """
mlflow models serve -m 'models:/MLDS/Production' -p 8001 --env-manager 'local' &
"""
get_ipython().system_raw(command)

In [14]:
import requests

In [15]:
data_request = X_test[:5].tolist()
display(data_request)

[[155.06645057344204,
  180.4996641791045,
  473.88399236499447,
  534.1337725118483],
 [142.57854857756303, 181.2399432624114, 473.9302877365304, 543.9558346430122],
 [147.4291861791115, 181.2381683305328, 474.01134920864206, 533.2130350990418],
 [162.87133979678893,
  171.99727480314965,
  474.0524742370154,
  563.4218205948696],
 [183.3498652755452, 167.34445433070866, 473.6963098986814, 563.4218205948696]]

In [16]:
r = requests.post("http://localhost:8001/invocations", json={"inputs": data_request})
print(r.text)

{"predictions": [[197.69068908691406], [180.2627716064453], [186.8044891357422], [209.0049591064453], [238.44163513183594]]}
